In [ ]:
from dotenv import load_dotenv
from crewai import Agent, Crew, Process, Task, LLM
from crewai.tools import tool
from langchain_google_vertexai import ChatVertexAI
import re
load_dotenv()

In [ ]:
llm = LLM(
model="vertex_ai/gemini-2.0-flash-thinking-exp-01-21", temperature=0.6)

In [ ]:
# Regex to Extract Code Snippet in triple backticks
flexible_pattern = re.compile(
    r"```python\s*(.*?)(```|$)",  # match until '```' or end-of-string
    re.DOTALL | re.IGNORECASE
)

def extract_code_block(raw_text: str) -> str:
    """
    Finds the substring after '```python' up to either the next triple backticks or end of string.
    Returns that as the code snippet. If not found, returns empty string.
    """
    match = flexible_pattern.search(raw_text)
    if match:
        code_part = match.group(1)
        # Remove leftover triple backticks just in case
        code_part = code_part.replace("```", "").strip()
        return code_part
    return ""

In [ ]:
# Data Visualization Agent
data_viz_agent = Agent(
    role="Data Visualization Agent",
    goal=(
        "Generate Python code using Plotly to visualize data based on user queries. "
        "Your code must be wrapped in triple backticks: ```python ... ``` and produce a 'fig' object."
    ),
    backstory=(
        "You are an expert data scientist. You have a local CSV file (already prepared). "
        "Use Plotly to create a figure (e.g. fig = px.bar(...)). "
        "Do not show or save the figure. Just produce the code snippet in triple backticks."
    ),
    tools=[],  # no extra tools needed for code generation
    llm=llm,
    verbose=True,
)

data_viz_task = Task(
    description="",
    expected_output="A snippet of valid Plotly code in triple backticks that creates a chart/visualization."
    "The data to use for chart is: {data}",
    agent=data_viz_agent,
)

viz_crew = Crew(
    agents=[data_viz_agent],
    tasks=[data_viz_task],
)

In [ ]:
data = {"a":2,
        "b":5,
        "c": 10
        
        }

In [ ]:
result = viz_crew.kickoff({"data":data})

In [19]:
import plotly.express as px

data = {'a': 2, 'b': 5, 'c': 10}
labels = list(data.keys())
values = list(data.values())

fig = px.bar(x=labels, y=values)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['a', 'b', 'c'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'AgUK', 'dtype': 'i1'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'x'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'y'}}}
})